In [3]:
import zipfile
import os, io
import numpy as np
from glob import glob
import pickle
import matplotlib.pyplot as plt

def unpickle(file):
    """
        Read downsampled 64*64 ImageNet data. Refer to: https://patrykchrabaszcz.github.io/Imagenet32/
        Return (dict): keys: ['data', 'labels', 'mean'], 
                    One file with validation data (“val_data”) contains python dictionary with fields 'data' and 'labels' (There is no 'mean' field)
    """
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

In [16]:
# Calculate mean value among all training data
for f in sorted(glob('/data/ljc/datasets/imagenet64/train' + '/*')):
    print(f)
    dic = unpickle(f)
    mean_val = dic['mean']
    break
mean_val /= 255.0

/data/ljc/datasets/imagenet64/train/train_data_batch_1


In [15]:
print(mean_val)

[118.45995643 119.56469453 120.35390702 ... 100.14106046  99.72706603
  99.14217975]


In [17]:
"""
    Read batch data and save each image to .npy files
"""

# read a line from the file and transform it into RGB image
def transform_to_rgb(data):
    red_channel = data[:4096].reshape(64, 64)
    green_channel = data[4096:8192].reshape(64, 64)
    blue_channel = data[8192:].reshape(64, 64)

    # merge channels
    rgb_image = np.dstack((red_channel, green_channel, blue_channel))
    return rgb_image
    
split = ['train', 'val']
sp = split[1]
folder = '/data/ljc/datasets/imagenet64/' + sp
out_folder = '/data/ljc/datasets/imagenet64/processed/' + sp
label_file = open(out_folder + '/labels.txt', 'w')

cnt = 0
for f in sorted(glob(folder + '/*')):
    print('Processing', f)
    
    dic = unpickle(f)
    
    for i in range(len(dic['data'])):
        data = dic['data'][i] / 255.0 - mean_val
        rgb_array = transform_to_rgb(data)       # scale to [0,1]. remove mean value, as suggested by https://patrykchrabaszcz.github.io/Imagenet32/
        np.save(f'{out_folder}/{cnt}.npy', rgb_array)
        
        label_file.write(str(dic['labels'][i]) + '\n')

        cnt += 1
        
label_file.close()

Processing /data/ljc/datasets/imagenet64/val/val_data
